#### SQL Tuning Exploration
In this notebook, we explore the effect that table indexes can have on query performance. Here, a series of five questions have been presented based on the ingested DBLP dataset. For each question, a link to the query logic is provided, which includes the commands used for the creation of table indexes. Results are displayed for query performance both before and after building indexes for comparison and commentary.

In [1]:
# Packages
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv
import logging
import re

In [2]:
# Constants
load_dotenv("../src/.env")
DB_USER = os.getenv("DB_USER")
DB_NAME = "dblp"
DB_PASSWORD = os.getenv("DB_PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")
CONNECT_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:{PORT}/{DB_NAME}"
SQL_CODE = "../src/sql_tuning_testing.sql"

# Settings
pd.set_option('display.max_colwidth', None)

# Engine
engine = create_engine(CONNECT_STRING)
Session = sessionmaker(engine)

In [3]:
# Parse query into a dictionary of dictionaries with keys (Question, Query, Indexes, Cleanup)
with open(SQL_CODE) as f:
    code = [i.split("--")[1:] for i in f.read().split("-- P")[1:]]

parsed_query = {f"P{k+1}":v for k,v in 
                enumerate([{re.sub(r"^/\*(\w+):[\S\s]*", r"\1",i):re.sub(r"\*/","",re.sub(r"^.*:\*/\n|^.*: ", "",i)) for i in j} for j in code])}

In [4]:
def indexing_experiment(question_num):
    """Helper function to iterate through parsed SQL code to run each query, evaluate performance with/without indices, and
    cleanup."""
    context = parsed_query[question_num]
    # Show Question
    print(context['Question'])

    # Show query result
    print("Query Result (First 5 rows)")
    display(pd.read_sql(context['Query'], engine).head())

    # Explain results before indexing
    print('EXPLAIN results - before indexing')
    before = pd.read_sql("EXPLAIN (ANALYZE,BUFFERS) " + context['Query'], engine)
    display(before)

    # Run indexing
    with Session() as session:
        session.execute(text(context['Indexing']))
        session.commit()

    # Re-run Explain
    print('EXPLAIN results - after indexing')
    after = pd.read_sql("EXPLAIN (ANALYZE,BUFFERS) " + context['Query'], engine)
    display(after)

    # Cleanup indexes
    with Session() as session:
        session.execute(text(context['Cleanup']))
        session.commit()

##### Question 1:

<a href="https://github.com/andrewcockerill/springboard_06_sql_tuning/blob/main/src/sql_tuning_testing.sql#L1">SQL Code</a>

We can see that by adding indices on <tt>year</tt> (function index using <tt>substr(year,3,1)</tt> to encode the decade) and <tt>booktitle</tt> to table, the cost decreases from 609916 to 416268, with a run time reduction of from ~1500 ms to ~900 ms.

In [5]:
indexing_experiment("P1")

Write a SQL Query to find all the conferences held in 2018 that have published at least 200 papers in a single decade.


Query Result (First 5 rows)


,booktitle
0,COLINS
1,ICL (2)
2,CBI (1)
3,ICSOC
4,FG


EXPLAIN results - before indexing


,QUERY PLAN
0,HashAggregate (cost=609916.11..609918.11 rows=200 width=8) (actual time=1556.229..1556.344 rows=1456 loops=1)
1,Group Key: inp.booktitle
2,Batches: 1 Memory Usage: 145kB
3,Buffers: shared hit=4763 read=160451
4,-> HashAggregate (cost=549876.72..608476.32 rows=115183 width=48) (actual time=1554.915..1555.835 rows=2691 loops=1)
5,"Group Key: inp.booktitle, substr(inp.year, 3, 1)"
6,Filter: (count(*) >= 200)
7,Planned Partitions: 8 Batches: 1 Memory Usage: 2065kB
8,Rows Removed by Filter: 3890
9,Buffers: shared hit=4763 read=160451


EXPLAIN results - after indexing


,QUERY PLAN
0,Unique (cost=416268.58..508132.91 rows=200 width=8) (actual time=966.182..977.967 rows=1456 loops=1)
1,Buffers: shared hit=79371 read=181817 written=10
2,-> Subquery Scan on paper_counts (cost=416268.58..507844.95 rows=115183 width=8) (actual time=966.182..977.685 rows=2691 loops=1)
3,Buffers: shared hit=79371 read=181817 written=10
4,-> Finalize GroupAggregate (cost=416268.58..506693.12 rows=115183 width=48) (actual time=966.181..977.543 rows=2691 loops=1)
5,"Group Key: inp.booktitle, (substr(inp.year, 3, 1))"
6,Filter: (count(*) >= 200)
7,Rows Removed by Filter: 3890
8,Buffers: shared hit=79371 read=181817 written=10
9,-> Gather Merge (cost=416268.58..496902.54 rows=691100 width=48) (actual time=966.170..975.595 rows=11201 loops=1)


##### Question 2:

<a href="https://github.com/andrewcockerill/springboard_06_sql_tuning/blob/main/src/sql_tuning_testing.sql#L42">SQL Code</a>

Here, indices were made for <tt>booktitle</tt> (conference) and <tt>journal</tt> to assist when doing filters on these columns. This resulted in a performance gain by reducing cost and execution times (1060 ms to 7 ms).

In [6]:
indexing_experiment('P2')

Write a SQL Query to find all the authors who published at least 10 PVLDB papers and at least 10 SIGMOD papers.


Query Result (First 5 rows)


,author
0,Andrew Eisenberg::Jim Melton
1,Michael Stonebraker
2,Karl Aberer
3,Michael J. Franklin
4,Arie Segev


EXPLAIN results - before indexing


,QUERY PLAN
0,HashSetOp Intersect (cost=217794.08..435616.22 rows=67 width=36) (actual time=1059.692..1065.038 rows=16 loops=1)
1,Buffers: shared hit=32466 read=309686
2,-> Append (cost=217794.08..435615.88 rows=134 width=36) (actual time=532.087..1065.023 rows=38 loops=1)
3,Buffers: shared hit=32466 read=309686
4,"-> Subquery Scan on ""*SELECT* 1"" (cost=217794.08..217922.18 rows=67 width=57) (actual time=532.086..538.274 rows=18 loops=1)"
...,...
61,"Filter: (journal = ANY ('{""ACM SIGMOD Anthology"",""ACM SIGMOD Digit. Rev."",""ACM SIGMOD Digit. Symp. Collect."",""FDT Bull. ACM SIGFIDET SIGMOD"",""SIGMOD Rec.""}'::text[]))"
62,Rows Removed by Filter: 1733055
63,Buffers: shared hit=192 read=88203
64,Planning Time: 0.482 ms


EXPLAIN results - after indexing


,QUERY PLAN
0,HashSetOp Intersect (cost=62576.55..113067.38 rows=67 width=36) (actual time=7.599..7.603 rows=16 loops=1)
1,Buffers: shared hit=286 read=347
2,-> Append (cost=62576.55..113067.04 rows=134 width=36) (actual time=4.362..7.593 rows=38 loops=1)
3,Buffers: shared hit=286 read=347
4,"-> Subquery Scan on ""*SELECT* 1"" (cost=62576.55..62579.72 rows=67 width=57) (actual time=4.362..4.976 rows=18 loops=1)"
5,Buffers: shared hit=73 read=301
6,-> HashAggregate (cost=62576.55..62579.05 rows=67 width=53) (actual time=4.362..4.974 rows=18 loops=1)
7,Group Key: inproceedings.author
8,Filter: (count(*) >= 10)
9,Batches: 1 Memory Usage: 1441kB


##### Question 3:

<a href="https://github.com/andrewcockerill/springboard_06_sql_tuning/blob/main/src/sql_tuning_testing.sql#L134">SQL Code</a>

In this case a functional index was made on <tt>substr(year,3,1)</tt> as done previously, though this only resulted in a modest performance gain from 21 ms to 15 ms.

In [7]:
indexing_experiment('P3')

Write a SQL Query to find the total number of conference publications for each decade, starting from 1970 and ending in 2019.


Query Result (First 5 rows)


,decade,count
0,0,13575
1,9,4450
2,7,355
3,1,26598
4,8,1202


EXPLAIN results - before indexing


,QUERY PLAN
0,HashAggregate (cost=3056.72..3057.61 rows=71 width=40) (actual time=20.294..20.295 rows=5 loops=1)
1,"Group Key: substr(proceedings.year, 3, 1)"
2,Batches: 1 Memory Usage: 24kB
3,Buffers: shared hit=1956
4,-> Seq Scan on proceedings (cost=0.00..3049.46 rows=1453 width=32) (actual time=0.009..15.531 rows=46180 loops=1)
5,"Filter: (substr(year, 3, 1) = ANY ('{7,8,9,0,1}'::text[]))"
6,Rows Removed by Filter: 11944
7,Buffers: shared hit=1956
8,Planning Time: 0.081 ms
9,Execution Time: 20.309 ms


EXPLAIN results - after indexing


,QUERY PLAN
0,HashAggregate (cost=1969.90..1970.79 rows=71 width=40) (actual time=14.446..14.447 rows=5 loops=1)
1,"Group Key: substr(proceedings.year, 3, 1)"
2,Batches: 1 Memory Usage: 24kB
3,Buffers: shared hit=1963 read=41
4,-> Bitmap Heap Scan on proceedings (cost=32.73..1962.63 rows=1453 width=32) (actual time=1.085..9.696 rows=46180 loops=1)
5,"Recheck Cond: (substr(year, 3, 1) = ANY ('{7,8,9,0,1}'::text[]))"
6,Heap Blocks: exact=1956
7,Buffers: shared hit=1963 read=41
8,-> Bitmap Index Scan on idx_decade_proceedings (cost=0.00..32.36 rows=1453 width=0) (actual time=0.928..0.928 rows=46180 loops=1)
9,"Index Cond: (substr(year, 3, 1) = ANY ('{7,8,9,0,1}'::text[]))"


##### Question 4:

<a href="https://github.com/andrewcockerill/springboard_06_sql_tuning/blob/main/src/sql_tuning_testing.sql#L159">SQL Code</a>

For this query, a full text approach was used in which a GIN index was created on the <tt>TO_TSVECTOR</tt> for the <tt>title</tt> columns in <tt>article</tt> and <tt>inproceedings</tt>. The rationale to use TS_QUERY in this case was to isolate just instances of the <tt>"data"</tt> token by itself. This resulted in a drop in cost and run time of the query.

In [8]:
indexing_experiment('P4')

Write a SQL Query to find the top 10 authors publishing in journals and conferences whose titles contain the word data.


Query Result (First 5 rows)


,author,papers
0,Alfredo Cuzzocrea,71
1,Complexity,23
2,Bhavani Thuraisingham,23
3,Charu C. Aggarwal::Philip S. Yu,23
4,Charu C. Aggarwal,22


EXPLAIN results - before indexing


,QUERY PLAN
0,Limit (cost=1651745.98..1651746.00 rows=10 width=61) (actual time=20681.233..20684.525 rows=10 loops=1)
1,"Buffers: shared hit=14626 read=157104, temp read=2794 written=2800"
2,-> Sort (cost=1651745.98..1651746.48 rows=200 width=61) (actual time=20681.232..20684.524 rows=10 loops=1)
3,Sort Key: (count(*)) DESC
4,Sort Method: top-N heapsort Memory: 25kB
5,"Buffers: shared hit=14626 read=157104, temp read=2794 written=2800"
6,-> Finalize GroupAggregate (cost=1651582.16..1651741.66 rows=200 width=61) (actual time=20240.596..20653.543 rows=316323 loops=1)
7,Group Key: article.author
8,"Buffers: shared hit=14626 read=157104, temp read=2794 written=2800"
9,-> Gather Merge (cost=1651582.16..1651737.66 rows=400 width=61) (actual time=20240.591..20580.750 rows=341716 loops=1)


EXPLAIN results - after indexing


,QUERY PLAN
0,Limit (cost=94170.74..94170.77 rows=10 width=61) (actual time=8728.286..8737.278 rows=10 loops=1)
1,"Buffers: shared hit=688 read=132718 written=15439, temp read=5450 written=7657"
2,-> Sort (cost=94170.74..94171.24 rows=200 width=61) (actual time=8728.285..8737.276 rows=10 loops=1)
3,Sort Key: (count(*)) DESC
4,Sort Method: top-N heapsort Memory: 25kB
5,"Buffers: shared hit=688 read=132718 written=15439, temp read=5450 written=7657"
6,-> Finalize GroupAggregate (cost=94115.75..94166.42 rows=200 width=61) (actual time=8299.985..8704.307 rows=316323 loops=1)
7,Group Key: article.author
8,"Buffers: shared hit=688 read=132718 written=15439, temp read=5450 written=7657"
9,-> Gather Merge (cost=94115.75..94162.42 rows=400 width=61) (actual time=8299.979..8633.233 rows=342437 loops=1)


##### Question 5:

<a href="https://github.com/andrewcockerill/springboard_06_sql_tuning/blob/main/src/sql_tuning_testing.sql#L204">SQL Code</a>

Here, indices were created for the <tt>booktitle</tt> column in both the <tt>proceedings</tt> and <tt>inproceedings</tt> tables to assist with the join between these two. This resulted in a savings of ~ 1000 ms.

In [9]:
indexing_experiment('P5')

Write a SQL query to find the names of all conferences, happening in June, where the proceedings contain more than 100 publications.


Query Result (First 5 rows)


,title,papers
0,"10th Asian Control Conference, ASCC 2015, Kota Kinabalu, Malaysia, May 31 - June 3, 2015",541
1,"10th IAPR International Conference on Pattern Recognition, Conference A: Computer Vision & Conference B Pattern recognition systems and applications, ICPR 1990, Atlantic City, NJ, USA, 16-21 June, 1990, Volume 1",194
2,"10th IAPR International Conference on Pattern Recognition, Conference C: image, speech, and signal processing, and Conference D: computer architecture for vision in pattern recognition, ICPR 1990, Atlantic City, NJ, USA, 16-21 June, 1990, Volume 2",121
3,"10th IEEE International Conference on Computer and Information Technology, CIT 2010, Bradford, West Yorkshire, UK, June 29-July 1, 2010",461
4,"10th IEEE International Conference on Control and Automation, ICCA 2013, Hangzhou, China, June 12-14, 2013",346


EXPLAIN results - before indexing


,QUERY PLAN
0,Limit (cost=160368.62..160375.31 rows=10 width=149) (actual time=1333.276..1387.362 rows=10 loops=1)
1,"Buffers: shared hit=2242 read=82541, temp read=15025 written=21392"
2,-> Finalize GroupAggregate (cost=160368.62..160433.43 rows=97 width=149) (actual time=1333.275..1387.360 rows=10 loops=1)
3,Group Key: a.title
4,Filter: (count(*) >= 100)
5,Rows Removed by Filter: 34
6,"Buffers: shared hit=2242 read=82541, temp read=15025 written=21392"
7,-> Gather Merge (cost=160368.62..160428.34 rows=291 width=149) (actual time=1332.817..1387.342 rows=45 loops=1)
8,Workers Planned: 1
9,Workers Launched: 1


EXPLAIN results - after indexing


,QUERY PLAN
0,Limit (cost=20493.59..53598.38 rows=10 width=149) (actual time=405.692..419.357 rows=10 loops=1)
1,Buffers: shared hit=2621 read=1718
2,-> GroupAggregate (cost=20493.59..341610.06 rows=97 width=149) (actual time=405.691..419.355 rows=10 loops=1)
3,Group Key: a.title
4,Filter: (count(*) >= 100)
5,Rows Removed by Filter: 34
6,Buffers: shared hit=2621 read=1718
7,-> Nested Loop (cost=20493.59..341579.15 rows=5455 width=141) (actual time=404.713..419.010 rows=3958 loops=1)
8,Buffers: shared hit=2621 read=1718
9,-> Gather Merge (cost=20485.03..20518.19 rows=291 width=156) (actual time=404.633..408.003 rows=48 loops=1)


In [10]:
engine.dispose()

##### Commentary on Caching Effects

In the process of running queries against tables, pages can be accessed on disk and are potentially available in the cache/buffer. In the <tt>EXPLAIN</tt> results above, this can be seen when a query pulls from memory (shared hit) versus from disk (read). Any instances where more pages can be accessed from memory will have a positive effect on query run times.